In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
cities = pd.read_csv("output_data/cities_df.csv")
cities

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Date
0,0,Brainerd,US,46.36,-94.20,75,65,78.80,78.80,1597017515
1,1,Ushuaia,AR,-54.80,-68.30,75,80,35.60,35.60,1597017661
2,2,Palivere,EE,58.97,23.89,12,81,63.00,63.00,1597017743
3,3,Alta Floresta,BR,-9.88,-56.09,12,33,76.41,76.41,1597017744
4,4,Ponta do Sol,PT,32.67,-17.10,40,73,73.40,73.40,1597017745
...,...,...,...,...,...,...,...,...,...,...
558,558,Stony Plain,CA,53.53,-114.00,1,34,69.01,69.01,1597018125
559,559,Wisconsin Rapids,US,44.38,-89.82,1,69,82.99,82.99,1597018352
560,560,Gorontalo,ID,0.54,123.06,74,80,80.10,80.10,1597018353
561,561,Tilichiki,RU,60.47,166.10,100,72,55.60,55.60,1597018354


In [3]:
#Humidity Heatmap
humidity = cities['Humidity'].astype(float)
max_humidity = humidity.max()
location = cities [["Latitude", "Longitude"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
cities_narrowed_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities ["Cloudiness"] == 0),:]
cities_narrowed_df = cities_narrowed_df.dropna(how='any')
cities_narrowed_df.reset_index(inplace=True)
cities_narrowed_df

,index,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Date
0,21,21,Conselice,IT,44.51,11.83,0,90,75.99,75.99,1597017764
1,52,52,Guiratinga,BR,-16.35,-53.76,0,36,71.98,71.98,1597017797
2,128,128,Guerrero Negro,MX,27.98,-114.06,0,49,78.64,78.64,1597017879
3,139,139,Abu Samrah,SY,35.30,37.18,0,81,70.65,70.65,1597017891
4,164,164,São João da Barra,BR,-21.64,-41.05,0,84,71.15,71.15,1597017918
5,196,196,Jardim,BR,-21.48,-56.14,0,32,71.96,71.96,1597017654
6,272,272,Alofi,NU,-19.06,-169.92,0,57,77.00,77.00,1597018038
7,276,276,Vitim,RU,59.45,112.56,0,50,70.18,70.18,1597018042
8,293,293,Chardara,KZ,41.25,67.97,0,40,74.35,74.35,1597018060
9,326,326,San Quintín,MX,30.48,-115.95,0,60,74.50,74.50,1597018096


In [18]:
hotel_df = cities_narrowed_df

hotel_list = []

for x in range(len(hotel_df)):
    lat = hotel_df.loc[x]['Latitude']
    lng = hotel_df.loc[x]['Longitude']
    
    params = {"location": f"{lat},{lng}",
             "types": "hotel",
             "radius": 5000,
             "key": g_key}
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params)
    rslt = response.json()
    #print(rslt)
    try:
        hotel_list.append(rslt['results'][0]['name'])
    except:
        print("missing result ... next")
        
hotel_df["Hotel Name"] = hotel_list
del hotel_df['index']
hotel_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Date,Hotel Name
0,21,Conselice,IT,44.51,11.83,0,90,75.99,75.99,1597017764,Conselice
1,52,Guiratinga,BR,-16.35,-53.76,0,36,71.98,71.98,1597017797,Guiratinga
2,128,Guerrero Negro,MX,27.98,-114.06,0,49,78.64,78.64,1597017879,Guerrero Negro
3,139,Abu Samrah,SY,35.30,37.18,0,81,70.65,70.65,1597017891,Suruj
4,164,São João da Barra,BR,-21.64,-41.05,0,84,71.15,71.15,1597017918,Grussaí


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))